# testing

In [2]:
!python COMP3221_BlockchainNode.py 5000 test/A.txt

^C


In [1]:
import socket
import importlib
import network
importlib.reload(network)

<module 'network' from 'f:\\uniIssue\\Comp3221\\A3\\A3\\network.py'>

In [6]:
address = '100.86.126.26:5000'
message = {
    "type": "transaction",
    "payload": {
        "sender": "a57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b",
        "message": "hello",
        "nonce": 0,
        "signature": "142e395895e0bf4e4a3a7c3aabf2f59d80c517d24bb2d98a1a24384bc7cb29c9d593ce3063c5dd4f12ae9393f3345174485c052d0f5e87c082f286fd60c7fd0c"
    }
}

network.send_transaction(address, message)


# After each testing

In [4]:
!netstat -aon | findstr :5000

In [6]:
!taskkill /PID 29216 /F

�ɹ�: ����ֹ PID Ϊ 29216 �Ľ��̡�


In [16]:
!del debug.txt